In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends import backend_pdf
from scipy.optimize import curve_fit
import scipy.stats as st
from scipy.odr import ODR, Model, Data, RealData
from helpers import *

## Configuration

In [ ]:
# list of PMTs at 0 voltage form HV files + disconnected pmts: 290
offPMTs = [1, 111, 143, 166, 192, 230, 238, 254, 222, 302, 309, 340, 353, 290 ]
offChs = PMTid_to_channel(offPMTs)

WESTonly = [ x for x in range(1,181) ]
chWESTonly = PMTid_to_channel(WESTonly)

EASTonly = [ x for x in range(181,361) ]
chEASTonly = PMTid_to_channel(EASTonly)

# drifting PMTs --> wait for full list
driftPMTsEAST = [ 282, 288, 296, 307, 310, 319, 346, 350]
driftPMTsWEST = [ 59, 63, 81, 93, 95, 142, 148, 169, 170, 179 ]
driftChsEAST = PMTid_to_channel(driftPMTsEAST)
driftChsWEST = PMTid_to_channel(driftPMTsWEST)

# PMTs not changed
nochangePMTs = [310, 282, 262, 248, 90]
nochangeChs = PMTid_to_channel(nochangePMTs)
print(offChs)

In [ ]:
def getDataframe(file, offPMTs):
       
    df=pd.read_csv(file, sep=',')
    
    # remove list of PMTs that are off
    channel_ids = PMTid_to_channel(offPMTs)
    df= df[~df['pmt'].isin(channel_ids)]
    
    return df

In [ ]:
# The fit function
def gaus(x,a,mean,sigma):
    return a*np.exp(-(x-mean)**2/(2*sigma**2))

def fitGaussian(x, BINSIZE=0.1, RMIN=0.0, RMAX=8.0, p0=(3., 0.5)):
    
    nbins=int((RMAX-RMIN)/BINSIZE)
    ys,edges = np.histogram(x,bins=nbins,range=(RMIN,RMAX))
    xs=np.array([edges[i]+0.5*(edges[i+1]-edges[i]) for i in range(len(ys))])

    param = [np.max(ys), p0[0], p0[1] ]
    bounds = np.array([(param[0]*0.5,param[0]*1.5),
              (param[1]*0.2,param[1]*1.3),
              (param[2]*0.1,param[2]*1.4)])
    
    param,pcov = curve_fit(gaus, xs, ys, p0=param, bounds=(bounds[:,0],bounds[:,1]) )
    eparam = np.diag(pcov)**0.5
    
    return xs, ys, param, eparam

In [ ]:
srcpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/amplitudedb/"
figpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/figs/"

# amplitude
#afile = srcpath + "bkgphamplitude_run10333_1692369304.csv" #+30V
#afile = srcpath + "bkgphamplitude_run10313_1692203499.csv"
#afile = srcpath + "bkgphamplitude_run10334_1692373315.csv" #-30V
#afile = srcpath + "bkgphamplitude_run10335_1692377987.csv" #-50V
#afile = srcpath + "bkgphamplitude_run10336_1692382296.csv" #-100V

afile = srcpath + "bkgphamplitude_run10890_1697390445.csv"

In [ ]:
run=getRun(afile)
adata = getDataFrame(afile, offPMTs)
xs, ys, param, eparam = fitGaussian(adata["q"].to_numpy(),BINSIZE=0.122, RMIN=0.0, RMAX=8.0, p0=(5., 0.5))
ys_fitted = gaus(xs, *param)

print(param[1],eparam[1])
print(param[2],eparam[2])

fig = plt.figure(figsize=(8,5))
nbins = int((8.0-0.0)/0.122)
plt.hist( adata["q"], histtype="step", lw=2, range=(0,8), bins=nbins, label="Run {}".format(run))

plt.plot( xs, ys_fitted, '-.', lw=3.0, label="Number of PMTs: {:d} \nMean: {:.4f} mV\nSigma: {:.4f} mV".format( np.sum(ys), param[1], param[2]))

plt.grid(alpha=0.5)
plt.legend(fontsize=12)
plt.xlabel("Amplitude [mV]",fontsize=12)
plt.ylabel("# PMTs",fontsize=12)
plt.savefig("figs/run{}_histo_amplitude.png".format(run),dpi=100)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,5))
plt.scatter(adata["pmt"],adata["q"],marker="o")

## latest equalization target
#plt.axhline(y=0.40, color="red", linestyle='dotted')

## KNOWN HV ISSUES
drift_dataWEST = adata[adata['pmt'].isin(driftChsWEST)]
drift_dataEAST = adata[adata['pmt'].isin(driftChsEAST)]
plt.scatter(drift_dataWEST["pmt"],drift_dataWEST["q"],marker="x",color="red", label="HV issues - now fixed (WEST)")
#plt.scatter(drift_dataEAST["pmt"],drift_dataEAST["q"],marker="x",color="magenta", label="HV issues (EAST)")

## NOT EQUALIZED
nochange_data = adata[adata['pmt'].isin(nochangeChs)]
plt.scatter(nochange_data['pmt'],nochange_data['q'],marker="^",color="green", label="Manual fit (PMT 90, ch 279)")

# MANUALLY EQUALIZED
# manual_data = data[data['pmt'].isin(manualChs)]
#plt.scatter(manual_data['pmt'],manual_data['q'],marker="^",color="violet", label="Manual adjustment")

#plt.xlim((180,360))
plt.ylim((0.0,8.0))
plt.xlabel("PMT channel ID")
plt.ylabel("Amplitude [mV]")
plt.legend()
plt.grid()

plt.savefig("run"+str(run)+"_scatter_amplitude.png", dpi=200)

## Amplitude vs gain

In [ ]:
def getAmplitudes( srcpath, filelist, offPMTs, start_values ):
    
    means = []
    sigmas = []
    for i, file in enumerate(filelist):
        
        f = srcpath + file
        run=getRun(f)
        adata = getDataFrame(f, offPMTs)
        
        xs, ys, param, eparam = fitGaussian(adata["q"].to_numpy(),p0=(start_values[i], 0.5))
        means.append(param[1])
        sigmas.append(param[2])
    
    return means, sigmas

def getGains( srcpath, filelist, offPMTs, start_values ):
    
    means = []
    sigmas = []
    for i, file in enumerate(filelist):
        
        f = srcpath + file
        run=getRun(f)
        adata = getDataFrame(f, offPMTs)
        
        xs, ys, param, eparam = fitGaussian(adata["q"].to_numpy(),BINSIZE=0.005,RMIN=0.0,RMAX=1.0,p0=(start_values[i], 0.05))
        means.append(param[1])
        sigmas.append(param[2])
    
    return means, sigmas

def line( p, x ):
    a, b = p
    return a*x+b

def getallForSingle( gain_file, amp_file , offPMTs, starts ):
    
    adata = getDataFrame(amp_file, offPMTs)
    gdata = getDataFrame(gain_file, offPMTs)
    
    xs, ys, param, eparam = fitGaussian(adata["q"].to_numpy(),p0=(starts[0], 0.5))
    amp, asigma = param[1], param[2]
    xs, ys, param, eparam = fitGaussian(gdata["q"].to_numpy(),BINSIZE=0.005,RMIN=0.0,RMAX=1.0,p0=(starts[1], 0.05))
    gain, gsigma = param[1], param[2]
    
    return gain, gsigma, amp, asigma

In [ ]:
voltages = [ "+30V", "Nominal", "-30V", "-50V", "-100V"]

In [ ]:
srcpath = "/icarus/data/users/mvicenzi/pmt-calibration/amplitudedb/"
filelist = ["bkgphamplitude_run10333_1692369304.csv", #+30V
            "bkgphamplitude_run10313_1692203499.csv",
            "bkgphamplitude_run10334_1692373315.csv", #-30V
            "bkgphamplitude_run10335_1692377987.csv", #-50V
            "bkgphamplitude_run10336_1692382296.csv", #-100V
           ]
start_values = [ 6.0, 5.0, 4.0, 4.0, 3.0]
amplitudes, asigmas = getAmplitudes(srcpath, filelist, offPMTs, start_values)

In [ ]:
print(amplitudes, asigmas)

In [ ]:
dbpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/"
files = [ "equalization_run10333_1692368269.csv", #+30V
          "equalization_run10313_1692203859.csv", #nominal 
          "equalization_run10334_1692374665.csv", #-30V
          "equalization_run10335_1692377717.csv", #-50V
          "equalization_run10336_1692384276.csv", #-100V
        ]
start_values = [0.6, 0.55, 0.48, 0.45, 0.38]
#exclude east + off
exclude = offPMTs + EASTonly
gains, gsigmas = getGains( dbpath, files, exclude, start_values )

In [ ]:
print( gains, gsigmas)

In [ ]:
gainpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/backgroundphotons_run9342_1672496767.csv"
amppath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/amplitudedb/bkgphamplitude_run9342_1672496767.csv"
starts = [3.5, 0.45]

g, sg, a, sa = getallForSingle( gainpath, amppath , offPMTs, starts )
print(g, sg, a, sa)

In [ ]:
fig = plt.figure(figsize=(9,6))
plt.errorbar(x=gains, y=amplitudes, xerr=gsigmas, yerr=asigmas, fmt="o")

data = RealData(x=gains, y=amplitudes, sx=gsigmas, sy=asigmas)
model = Model(line)
odr = ODR(data, model, beta0=[1,1])
odr.set_job(fit_type=2)
output = odr.run()

xn = np.linspace(np.min(gains)-0.05,np.max(gains),200)
yn = line(output.beta, xn)
param = output.beta
plt.plot(xn,yn,color="red",lw=2, label="Linear fit: ax+b\na: {:.2f} mV/10$^7$ elec.\nb: {:.2f} mV".format(param[0],param[1]))

plt.axvline(x=g,linestyle="dotted",label="Run 9342 (RUN 2)\nMean Gain", color="green")
plt.axvspan(xmin=g-sg,xmax=g+sg,color="green",alpha=0.1)

plt.axhline(y=a,linestyle="dotted",label="Run 9342 (RUN 2)\nMean PE amplitude", color="orange")
plt.axhspan(ymin=a-sa,ymax=a+sa,color="orange",alpha=0.1)

for i,v in enumerate(voltages):
    plt.annotate( v, xy=(gains[i],amplitudes[i]),textcoords="offset points", xytext=(-25,22), ha='center')

plt.grid(alpha=0.5,linestyle="dashed")
plt.xlabel("Gain [10$^7$ electrons]", fontsize=14)
plt.ylabel("Single PE amplitude [mV]", fontsize=14)
plt.legend(fontsize=12)

plt.savefig("figs/WEST_AmplitudeVsGain.png", dpi=500)
plt.show()